# Javascript Object Notation (JSON)

- Es una notación común para data web
- No tabular:
     - Registros no tienen todos el mismo conjunto de atributos
- Datos se encuentran organizados en colecciones de objetos
- Objectos son colecciones de atributos clave : valor
- Json nested: objectos se encuentran anidados

#### Tipos de Archivo Json

- **Record Orientation**: JSON más común
- **Column Orientation**: Uso de espación más eficiente que en record Orientation
- **Specifying Orientation**: Divide orientation data

<img src='https://docs.oracle.com/en/database/oracle/oracle-database/21/lnoci/img/json_doc.png'>

In [0]:
%run ./Includes/Classroom-Setup

### ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) Lectura de archivos JSON

Leer de JSON con el método `json` de DataFrameReader y la opción de esquema de inferencia

In [0]:
eventsJsonPath = "/FileStore/events-500k.json"

eventsDF = (spark.read
  .option("inferSchema", True)
  .json(eventsJsonPath))

eventsDF.printSchema()

root
 |-- device: string (nullable = true)
 |-- ecommerce: struct (nullable = true)
 |    |-- purchase_revenue_in_usd: double (nullable = true)
 |    |-- total_item_quantity: long (nullable = true)
 |    |-- unique_items: long (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_previous_timestamp: long (nullable = true)
 |-- event_timestamp: long (nullable = true)
 |-- geo: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- coupon: string (nullable = true)
 |    |    |-- item_id: string (nullable = true)
 |    |    |-- item_name: string (nullable = true)
 |    |    |-- item_revenue_in_usd: double (nullable = true)
 |    |    |-- price_in_usd: double (nullable = true)
 |    |    |-- quantity: long (nullable = true)
 |-- traffic_source: string (nullable = true)
 |-- user_first_touch_timestamp: long (nullable = true)

Lea datos más rápido creando un `StructType` con los nombres de esquema y tipos de datos

In [0]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, LongType, StringType, StructType, StructField

userDefinedSchema = StructType([
  StructField("device", StringType(), True),
  StructField("ecommerce", StructType([
    StructField("purchaseRevenue", DoubleType(), True),
    StructField("total_item_quantity", LongType(), True),
    StructField("unique_items", LongType(), True)
  ]), True),
  StructField("event_name", StringType(), True),
  StructField("event_previous_timestamp", LongType(), True),
  StructField("event_timestamp", LongType(), True),
  StructField("geo", StructType([
    StructField("city", StringType(), True),
    StructField("state", StringType(), True)
  ]), True),
  StructField("items", ArrayType(
    StructType([
      StructField("coupon", StringType(), True),
      StructField("item_id", StringType(), True),
      StructField("item_name", StringType(), True),
      StructField("item_revenue_in_usd", DoubleType(), True),
      StructField("price_in_usd", DoubleType(), True),
      StructField("quantity", LongType(), True)
    ])
  ), True),
  StructField("traffic_source", StringType(), True),
  StructField("user_first_touch_timestamp", LongType(), True),
  StructField("user_id", StringType(), True)
])

eventsDF = (spark.read
  .schema(userDefinedSchema)
  .json(eventsJsonPath))

## Descargar Archivos de Databricks

In [0]:
# mostrar los archivos que se encuentren en una ruta especifica
dbutils.fs.ls('dbfs:/FileStore')



Out[1]: [FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0)]

In [0]:
# Para copiar tal carpeta a la ruta de destino de forma recursiva (TRUE)
dbutils.fs.cp('src', 'dts', True)


In [0]:
# Eliminar archivos de una ruta especifica
dbutils.fs.rm("/FileStore/tables/",True)

Out[3]: True

In [0]:
# Ayuda con los comandos
dbutils.fs.help()

dbutils.fs provides utilities for working with FileSystems. Most methods in
this package can take either a DBFS path (e.g., "/foo" or "dbfs:/foo"), or
another FileSystem URI.

For more info about a method, use dbutils.fs.help("methodName") .

In notebooks, you can also use the %fs shorthand to access DBFS. The %fs shorthand maps
straightforwardly onto dbutils calls. For example, "%fs head --maxBytes=10000 /file/path"
translates into "dbutils.fs.head("/file/path", maxBytes = 10000)".
 fsutils cp(from: String, to: String, recurse: boolean = false): boolean -> Copies a file or directory, possibly across FileSystems head(file: String, maxBytes: int = 65536): String -> Returns up to the first 'maxBytes' bytes of the given file as a String encoded in UTF-8 ls(dir: String): Seq -> Lists the contents of a directory mkdirs(dir: String): boolean -> Creates the given directory if it does not exist, also creating any necessary parent directories mv(from: String, to: String, recurse: boolean = false): boolean -> Moves a file or directory, possibly across FileSystems put(file: String, contents: String, overwrite: boolean = false): boolean -> Writes the given String out to a file, encoded in UTF-8 rm(dir: String, recurse: boolean = false): boolean -> Removes a file or directory mount mount(source: String, mountPoint: String, encryptionType: String = "", owner: String = null, extraConfigs: Map = Map.empty[String, String]): boolean -> Mounts the given source directory into DBFS at the given mount point mounts: Seq -> Displays information about what is mounted within DBFS refreshMounts: boolean -> Forces all machines in this cluster to refresh their mount cache, ensuring they receive the most recent information unmount(mountPoint: String): boolean -> Deletes a DBFS mount point

/dbfs/mnt/training/ecommerce/events/events-500k.json

<!-- Estructura para el link de opiar de archivos -->
https://community.cloud.databricks.com/files/[path-file]?o=[id]

Ejemplo de path tiene http....com/files/[path-file]
Para el caso de [id] tienes que ver el link cuando ingreses o estes logeado en data bricks, por ejemplo el link qeu tengo el id = 3574798299160009#
https://community.cloud.databricks.com/?o=[3574798299160009]#notebook/3539616317455911/command/3539616317455921

https://community.cloud.databricks.com/files/events-500k.json/part-00000-tid-309888144738233288-fab86c62-ff9f-4176-98c6-587f95ee9066-2365-1-c000.json?o=8507771068099703

## Laboratorio
------------------

- Leer archivo flight-data.json según:
  - Infiere esquema
  - Generar esquema de lectura

In [1]:
# Only when is Local
import findspark

findspark.init()
findspark.find()

'E:\\LibreriasPython\\spark-3.1.2-bin-hadoop2.7\\python\\pyspark'

In [4]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ManagementJSON').getOrCreate()

In [5]:
flightJsonPath = "../data/2015-summary.json"

flightDF = (spark.read
  .option("inferSchema", True)
  .json(flightJsonPath))

flightDF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
flightDF.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [7]:
from pyspark.sql.types import ArrayType, DoubleType, IntegerType, LongType, StringType, StructType, StructField

userDefinedSchema = StructType([
  StructField("DEST_COUNTRY_NAME", StringType(), True),
  StructField("ORIGIN_COUNTRY_NAME", StringType(),True),
  StructField("count", IntegerType(), True),

])

flightDF = (spark.read
  .schema(userDefinedSchema)
  .json(flightJsonPath))

flightDF.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [8]:
flightDF.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows

